In [0]:
import numpy as np
import pandas as pd
import os
import io
import matplotlib.pyplot as plt
import cv2
import time
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import cohen_kappa_score
from sklearn.ensemble import BaggingClassifier
from scipy import stats
from joblib import dump,load

In [0]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

In [0]:
%%time
!cp -r /content/gdrive/My\ Drive/MURA_files/data/MURA-v1.1.zip MURA-v1.1.zip
!unzip -q MURA-v1.1.zip
!cp -r /content/gdrive/My\ Drive/MURA_files/data/processed processed

In [0]:
!ls processed

In [0]:
mura_prefix = '/content/'

In [0]:
def process_img(path):
  img_name = os.path.join(mura_prefix, path)   
  image = cv2.imread(img_name, 0)
  ## thresholding
  image = cv2.adaptiveThreshold(image,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,11,2)  
  image = cv2.resize(image, (227, 227))
  return image.reshape(1, 227 * 227)

In [0]:
def initialize(data):
  X = np.zeros(shape=(len(data), 227 * 227))
  y = data.loc[:, 'result']
  for r in range(len(data)):
    img = process_img(data.loc[r, 'path'])
    X[r, :] = img
    return(X,y)

In [0]:
def initialize_val(val_dat):
  X_val = np.zeros(shape=(len(val_dat), 227 * 227))
  y_val = val_dat.loc[:, 'result']
  for r in range(len(val_dat)):
    img = process_img(val_dat.loc[r, 'path'])
    X_val[r, :] = img
    return(X_val,y_val)

In [0]:
parts=['finger','humerus','forearm','shoulder','elbow','hand','wrist']

In [0]:
%%time
for i in parts:
  valid_csv='/content/processed/valid_'+i+'.csv'
  val_dat = pd.read_csv(valid_csv)
  X_val,y_val=initialize_val(val_dat)
  X_val=X_val/255
  path='/content/gdrive/My Drive/MURA_files/models/svm_'+i+'.joblib'
  model=load(path)
  print(model.score(X_val,y_val))
  pred_val=model.predict(X_val)
  kappa_score=cohen_kappa_score(pred_val,y_val)
  print('validation kappa score:',kappa_score)
  